In [14]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load and parse the XML file
tree = ET.parse('zug.xml')
root = tree.getroot()


In [15]:


# Initialize lists to store data
data = []

# Iterate over each 's' element in the XML
for s in root.findall('.//s'):
    s_id = s.get('id')
    s_eva = s.get('eva')
    
    # Extract 'm' element data
    m = s.find('m')
    if m is not None:
        m_id = m.get('id')
        m_t = m.get('t')
        m_from = m.get('from')
        m_to = m.get('to')
        m_cat = m.get('cat')
        m_ts = m.get('ts')
        m_pr = m.get('pr')
    else:
        m_id = m_t = m_from = m_to = m_cat = m_ts = m_pr = None
    
    # Extract 'ar' element data
    ar = s.find('ar')
    if ar is not None:
        ar_ct = ar.get('ct')
        ar_l = ar.get('l')
    else:
        ar_ct = ar_l = None
    
    # Extract 'dp' element data
    dp = s.find('dp')
    if dp is not None:
        dp_ct = dp.get('ct')
        dp_l = dp.get('l')
    else:
        dp_ct = dp_l = None
    
    # Append the extracted data to the list
    data.append([s_id, s_eva, m_id, m_t, m_from, m_to, m_cat, m_ts, m_pr, ar_ct, ar_l, dp_ct, dp_l])

# Define column names
columns = ['s_id', 's_eva', 'm_id', 'm_t', 'm_from', 'm_to', 'm_cat', 'm_ts', 'm_pr', 'ar_ct', 'ar_l', 'dp_ct', 'dp_l']

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

# Display the DataFrame to ensure it loaded correctly
print(df.head())


                                 s_id    s_eva      m_id   m_t      m_from  \
0   -2966325542119918826-2406181352-2  8000225  r2215104     h  2406181200   
1   -4024876283071378009-2406181239-6  8000225      None  None        None   
2   8999712939099255683-2406181305-15  8000225  r2215104     h  2406181200   
3  6024700710364942789-2406181542-100  8000225      None  None        None   
4     470377477421835666-2406181153-8  8000225      None  None        None   

         m_to                           m_cat        m_ts  m_pr       ar_ct  \
0  2406182200  Störung. (Quelle: zuginfo.nrw)  2406180950     1  2406181416   
1        None                            None        None  None  2406181333   
2  2406182200  Störung. (Quelle: zuginfo.nrw)  2406180950     1  2406181453   
3        None                            None        None  None        None   
4        None                            None        None  None  2406181320   

   ar_l       dp_ct  dp_l  
0  RE15  2406181417  RE15  


In [16]:
# Convert time columns to datetime
df['m_from'] = pd.to_datetime(df['m_from'], format='%y%m%d%H%M')
df['m_to'] = pd.to_datetime(df['m_to'], format='%y%m%d%H%M')
df['ar_ct'] = pd.to_datetime(df['ar_ct'], format='%y%m%d%H%M')
df['dp_ct'] = pd.to_datetime(df['dp_ct'], format='%y%m%d%H%M')

# Find the duration of events
df['duration'] = df['m_to'] - df['m_from']

# Analyze delays
df['delay'] = df['dp_ct'] - df['ar_ct']

In [18]:
df.head()

,s_id,s_eva,m_id,m_t,m_from,m_to,m_cat,m_ts,m_pr,ar_ct,ar_l,dp_ct,dp_l,duration,delay
0,-2966325542119918826-2406181352-2,8000225,r2215104,h,2024-06-18 12:00:00,2024-06-18 22:00:00,Störung. (Quelle: zuginfo.nrw),2406180950,1,2024-06-18 14:16:00,RE15,2024-06-18 14:17:00,RE15,0 days 10:00:00,0 days 00:01:00
1,-4024876283071378009-2406181239-6,8000225,None,None,NaT,NaT,None,None,None,2024-06-18 13:33:00,1,2024-06-18 13:41:00,1,NaT,0 days 00:08:00
2,8999712939099255683-2406181305-15,8000225,r2215104,h,2024-06-18 12:00:00,2024-06-18 22:00:00,Störung. (Quelle: zuginfo.nrw),2406180950,1,2024-06-18 14:53:00,RE15,2024-06-18 14:54:00,RE15,0 days 10:00:00,0 days 00:01:00
3,6024700710364942789-2406181542-100,8000225,None,None,NaT,NaT,None,None,None,NaT,None,NaT,None,NaT,NaT
4,470377477421835666-2406181153-8,8000225,None,None,NaT,NaT,None,None,None,2024-06-18 13:20:00,56,2024-06-18 13:27:00,56,NaT,0 days 00:07:00


In [19]:
# Summary statistics
df.describe()

,m_from,m_to,ar_ct,dp_ct,duration,delay
count,35,35,77,77,35,73
mean,2024-06-18 09:34:32.571428608,2024-06-18 19:34:30.857142528,2024-06-18 11:54:02.337662208,2024-06-18 12:07:41.298701312,0 days 09:59:58.285714285,0 days 00:04:55.890410958
min,2024-06-18 00:09:00,2024-06-18 09:28:00,2024-06-18 04:33:00,2024-06-18 04:42:00,0 days 02:00:00,0 days 00:01:00
25%,2024-06-18 07:28:00,2024-06-18 14:30:00,2024-06-18 08:49:00,2024-06-18 08:52:00,0 days 06:32:00,0 days 00:01:00
50%,2024-06-18 12:00:00,2024-06-18 22:00:00,2024-06-18 11:43:00,2024-06-18 12:10:00,0 days 10:00:00,0 days 00:03:00
75%,2024-06-18 12:00:00,2024-06-18 22:00:00,2024-06-18 14:51:00,2024-06-18 14:54:00,0 days 10:00:00,0 days 00:08:00
max,2024-06-18 12:05:00,2024-06-19 03:41:00,2024-06-19 00:11:00,2024-06-19 00:12:00,1 days 03:32:00,0 days 00:12:00
std,NaN,NaN,NaN,NaN,0 days 05:40:38.354009602,0 days 00:03:47.008538853


In [ ]:
# Check for missing values
df.isnull().sum()

s_id         0
s_eva        0
m_id        62
m_t         62
m_from      62
m_to        62
m_cat       62
m_ts        62
m_pr        62
ar_ct       20
ar_l        38
dp_ct       20
dp_l        39
duration    62
delay       24
dtype: int64